# K-means iterativo

## Spiegazione codice:

clustering iterativo bilanciato basato su K-means, progettato per dividere i clienti in zone geografiche da servire con un vincolo temporale forte (8 ore di turno massimo per corriere). Ecco come funziona dettagliatamente:

1. setup:
- Riceve un DataFrame con i punti di consegna (colonne: location_id, lat, lon).
- Standardizza le coordinate geografiche (latitudine e longitudine) per rendere più efficace il clustering K-means.
- Calcola un numero iniziale di cluster K adattivo sulla base delle dimensioni del dataset (es. un cluster ogni ~200 punti).

2. L’algoritmo si basa su una procedura iterativa (massimo max_iterations):
- Esegue K-means con il K corrente per assegnare ogni punto a un cluster.
- Costruisce un dizionario di cluster con le liste di location_id assegnati a ciascun cluster.
- Per ogni cluster, calcola le statistiche di performance tramite la funzione single_cluster_stats_with_cache.
- Ottiene il tempo medio massimo di consegna (mean_minutes) per ogni cluster.
- Individua i cluster il cui tempo medio massimo supera il vincolo di 8 ore (+ una tolleranza configurabile).
- Se nessun cluster supera il limite, si ferma con successo.
- Tiene traccia della migliore soluzione finora trovata (minore numero di cluster problematici).

3. Riclusterizzazione Intelligente:
- Calcola, per ogni cluster problematico, quanti sotto-cluster creare, usando una funzione basata sul rapporto tra tempo stimato e limite orario, e la dimensione del cluster.
- Estrae i punti dei cluster problematici dal dataset.
- Riclusterizza solo quei punti in nuovi cluster più piccoli, aggiornando le assegnazioni.
- Incrementa K in modo dinamico per suddividere meglio i cluster che superano il vincolo.

4. Condizioni di Uscita: 
- Tutti i cluster rispettano il vincolo di tempo o sono entro la tolleranza.
- Mancano miglioramenti per 3 iterazioni consecutive.
- Viene raggiunto il numero massimo di iterazioni.

5. Parallelismo e Memoria:
- Il calcolo delle performance per ogni cluster è eseguito in parallelo usando ThreadPoolExecutor, sfruttando al massimo i core CPU disponibili.
- Implementa una cache persistente su disco per salvare i risultati già calcolati, evitando di rifare i calcoli pesanti per cluster già analizzati.

# K-means iterativo v2

## Descrizione e differenze col precedente

1. Merge di cluster piccoli confinanti
- Trova cluster confinanti usando triangolazione di Delaunay
- Controlla se la somma dei tempi medi giornalieri non supera 8 ore
- Unisce automaticamente cluster troppo piccoli (< 15 punti) se compatibili

2. Ottimizzazioni di velocità
- Usa calc_clusters_stats invece di single_cluster_stats_with_cache per evitare nested parallelism
- Controlli merge in parallelo per tutte le coppie candidate
- Cache più efficiente con informazioni per weekday

3. Flusso ottimizzato
- K-means tradizionale
- Merge cluster piccoli confinanti
- Identifica cluster problematici (> 8 ore)
- Riclusterizza solo quelli problematici
- Ripete fino a convergenza

## imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing as mp
import time
import os
import pickle
from scipy.spatial import Delaunay
from itertools import combinations
# import del notebook per il calcolo del routing
import import_ipynb
import performance_calc as pc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650547 entries, 0 to 650546
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   location_id     650547 non-null  int64  
 1   lat             650547 non-null  float64
 2   lon             650547 non-null  float64
 3   quantity        650547 non-null  int64  
 4   delivery_date   650547 non-null  object 
 5   window_start_0  639992 non-null  object 
 6   window_end_0    639992 non-null  object 
 7   is_event        650547 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 39.7+ MB
❌ Valori non convertiti in 'delivery_date':
[]

❌ Valori non convertibili in booleani in 'is_event':
[]
❌ Valori non convertiti in window_start_0:
[None]

❌ Valori non convertiti in window_end_0:
[None]


<string>:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ True False False ...  True  True  True]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


<class 'pandas.core.frame.DataFrame'>
Index: 63733 entries, 0 to 650546
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   location_id     63733 non-null  int64         
 1   lat             63733 non-null  float64       
 2   lon             63733 non-null  float64       
 3   quantity        63733 non-null  int64         
 4   delivery_date   63733 non-null  datetime64[ns]
 5   window_start_0  63590 non-null  object        
 6   window_end_0    63590 non-null  object        
 7   is_event        63733 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 4.4+ MB
Punti di consegna unici trovati: 3766
Dopo l'eliminazione di punti troppo distanti, sono rimasti 3764 punti di consegna
delivery points esempio:
   location_id        lat        lon
0         2884  45.710720  10.047550
1         2885  45.670435   9.935415
2         2886  45.671596   9.931096
3 

## k=50 full

In [2]:
class OptimizedBalancedClustering:
    def __init__(self, 
                 max_shift_time_min: int = 480,
                 n_cores: int = None,
                 cache_dir: str = "./cluster_cache"):
        
        self.max_shift_time_min = max_shift_time_min
        self.n_cores = n_cores or max(1, mp.cpu_count() - 1)
        self.cache_dir = cache_dir
        self.global_cache = {}
        
        os.makedirs(cache_dir, exist_ok=True)
        print(f"🚀 Inizializzato con {self.n_cores} core CPU")
    
    def _cache_key(self, location_ids):
        return hash(tuple(sorted(location_ids)))
    
    def _load_cache(self):
        cache_file = os.path.join(self.cache_dir, "cluster_performance_cache.pkl")
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'rb') as f:
                    self.global_cache = pickle.load(f)
                print(f"📂 Caricata cache con {len(self.global_cache)} entries")
            except:
                self.global_cache = {}
    
    def _save_cache(self):
        cache_file = os.path.join(self.cache_dir, "cluster_performance_cache.pkl")
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(self.global_cache, f)
            print(f"💾 Salvata cache con {len(self.global_cache)} entries")
        except Exception as e:
            print(f"⚠️ Errore salvataggio cache: {e}")
    
    def _find_neighboring_clusters_delaunay(self, delivery_points):
        """
        Trova cluster confinanti usando triangolazione di Delaunay per ottimizzare velocità
        """
        try:
            coords = delivery_points[['lat', 'lon']].values
            tri = Delaunay(coords)
            
            neighbors_dict = {}
            for cluster_id in delivery_points['cluster'].unique():
                neighbors_dict[cluster_id] = set()
            
            # Per ogni triangolo, trova cluster coinvolti
            for triangle in tri.simplices:
                clusters_in_triangle = delivery_points.iloc[triangle]['cluster'].unique()
                
                if len(clusters_in_triangle) > 1:
                    for i, cluster1 in enumerate(clusters_in_triangle):
                        for j, cluster2 in enumerate(clusters_in_triangle):
                            if i != j:
                                neighbors_dict[cluster1].add(cluster2)
            
            # Converte set in list
            neighbors_dict = {k: list(v) for k, v in neighbors_dict.items()}
            return neighbors_dict
            
        except Exception as e:
            print(f"⚠️ Errore calcolo neighbors: {e}")
            return {}
    
    def _can_merge_clusters(self, times1, times2, threshold=480):
        """
        Verifica se due cluster possono essere uniti controllando che la somma 
        dei tempi medi per ogni giorno non superi il limite
        """
        if len(times1) != len(times2):
            return False, None
            
        sum_times = np.array(times1) + np.array(times2)
        return np.all(sum_times <= threshold), sum_times.tolist()
    
    def _parallel_cluster_evaluation_optimized(self, cluster_dict, time_limit=3):
        """
        Versione ottimizzata che usa calc_clusters_stats per evitare nested parallelism
        """
        print(f"🔄 Calcolo performance di {len(cluster_dict)} cluster con calc_clusters_stats...")
        start_time = time.time()
        
        # Filtra cluster non vuoti
        valid_clusters = {cid: locs for cid, locs in cluster_dict.items() if len(locs) > 0}
        
        if not valid_clusters:
            return {}
        
        # Usa calc_clusters_stats che è più efficiente per molti cluster
        clusters_list = list(valid_clusters.values())
        
        try:
            performance_df = pc.calc_clusters_stats(
                clusters=clusters_list,
                time_limit=time_limit,
                parallel=True,
                max_workers=self.n_cores,
                verbose=False
            )
            
            # Converti risultati in formato compatibile
            results = {}
            cluster_ids = list(valid_clusters.keys())
            
            for i, cluster_id in enumerate(cluster_ids):
                cluster_data = performance_df[performance_df['cluster'] == f'Cluster {i+1}']
                
                if not cluster_data.empty:
                    max_time = cluster_data['mean_minutes'].max()
                    avg_time = cluster_data['mean_minutes'].mean()
                    
                    # Estrai tempi per giorno della settimana per merge analysis
                    weekday_times = []
                    for weekday in ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì']:
                        weekday_data = cluster_data[cluster_data['weekday'] == weekday]
                        if not weekday_data.empty:
                            weekday_times.append(weekday_data['mean_minutes'].iloc[0])
                        else:
                            weekday_times.append(0)
                    
                    # Aggiungi cache entry
                    cache_key = self._cache_key(valid_clusters[cluster_id])
                    result = {
                        'max_time': max_time,
                        'avg_time': avg_time,
                        'cluster_size': len(valid_clusters[cluster_id]),
                        'feasible': max_time <= self.max_shift_time_min + 30,
                        'weekday_times': weekday_times  # Nuovo campo per merge analysis
                    }
                    
                    self.global_cache[cache_key] = result
                    results[cluster_id] = result
                else:
                    results[cluster_id] = {
                        'max_time': float('inf'),
                        'avg_time': float('inf'),
                        'cluster_size': len(valid_clusters[cluster_id]),
                        'feasible': False,
                        'weekday_times': [float('inf')] * 5
                    }
            
            elapsed = time.time() - start_time
            print(f"✅ Completata valutazione ottimizzata in {elapsed:.1f}s")
            return results
            
        except Exception as e:
            print(f"⚠️ Errore in calc_clusters_stats: {e}")
            # Fallback al metodo originale
            return self._parallel_cluster_evaluation_fallback(cluster_dict, time_limit)
    
    def _parallel_cluster_evaluation_fallback(self, cluster_dict, time_limit=3):
        """Metodo fallback originale"""
        print(f"🔄 Fallback: calcolo performance di {len(cluster_dict)} cluster...")
        start_time = time.time()
        
        jobs = [(cid, loc_ids, time_limit) for cid, loc_ids in cluster_dict.items() if len(loc_ids) > 0]
        results = {}
        
        with ThreadPoolExecutor(max_workers=self.n_cores) as executor:
            future_to_cluster = {
                executor.submit(self._compute_cluster_performance_cached, loc_ids, time_limit): cid 
                for cid, loc_ids, time_limit in jobs
            }
            
            completed = 0
            for future in as_completed(future_to_cluster):
                cluster_id = future_to_cluster[future]
                try:
                    result = future.result()
                    results[cluster_id] = result
                    completed += 1
                    
                    if completed % 10 == 0:
                        elapsed = time.time() - start_time
                        print(f"  📊 Completati {completed}/{len(jobs)} cluster in {elapsed:.1f}s")
                        
                except Exception as e:
                    print(f"⚠️ Errore cluster {cluster_id}: {e}")
                    results[cluster_id] = {
                        'max_time': float('inf'),
                        'avg_time': float('inf'), 
                        'cluster_size': 0,
                        'feasible': False,
                        'weekday_times': [float('inf')] * 5
                    }
        
        elapsed = time.time() - start_time
        print(f"✅ Completata valutazione fallback in {elapsed:.1f}s")
        return results
    
    def _compute_cluster_performance_cached(self, location_ids, time_limit=3):
        cache_key = self._cache_key(location_ids)
        
        if cache_key in self.global_cache:
            return self.global_cache[cache_key]
        
        try:
            stats_df, _ = pc.single_cluster_stats_with_cache(
                cluster_location_ids=location_ids,
                time_limit=time_limit,
                verbose=False,
                max_workers=1
            )
            
            if stats_df is not None and not stats_df.empty:
                max_time = stats_df['mean_minutes'].max()
                avg_time = stats_df['mean_minutes'].mean()
                
                # Estrai tempi per weekday per merge analysis
                weekday_times = []
                for weekday in ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì']:
                    weekday_data = stats_df[stats_df['weekday'] == weekday]
                    if not weekday_data.empty:
                        weekday_times.append(weekday_data['mean_minutes'].iloc[0])
                    else:
                        weekday_times.append(0)
                
                result = {
                    'max_time': max_time,
                    'avg_time': avg_time,
                    'cluster_size': len(location_ids),
                    'feasible': max_time <= self.max_shift_time_min + 30,
                    'weekday_times': weekday_times
                }
            else:
                result = {
                    'max_time': float('inf'),
                    'avg_time': float('inf'),
                    'cluster_size': len(location_ids),
                    'feasible': False,
                    'weekday_times': [float('inf')] * 5
                }
            
            self.global_cache[cache_key] = result
            return result
            
        except Exception as e:
            print(f"⚠️ Errore calcolo cluster {len(location_ids)} punti: {e}")
            return {
                'max_time': float('inf'),
                'avg_time': float('inf'),
                'cluster_size': len(location_ids),
                'feasible': False,
                'weekday_times': [float('inf')] * 5
            }
    
    def _merge_small_neighboring_clusters(self, cluster_dict, cluster_results, neighbors_dict, 
                                         points_df, min_cluster_size=15, verbose=True):
        """
        Unisce cluster piccoli confinanti se la somma dei tempi non supera il limite
        """
        if verbose:
            print(f"🔗 Analisi merge cluster piccoli (< {min_cluster_size} punti)...")
        
        merged = set()
        new_cluster_dict = cluster_dict.copy()
        merge_count = 0
        
        # Identifica cluster piccoli
        small_clusters = [c for c, locs in cluster_dict.items() 
                         if len(locs) <= min_cluster_size and c in cluster_results]
        
        if verbose:
            print(f"  📊 Trovati {len(small_clusters)} cluster piccoli da analizzare")
        
        # Prepara dati per controllo parallelo
        merge_candidates = []
        
        for c in small_clusters:
            if c in merged or c not in neighbors_dict:
                continue
                
            c_times = cluster_results[c].get('weekday_times', [])
            if not c_times or any(t == float('inf') for t in c_times):
                continue
            
            for n in neighbors_dict[c]:
                if n in merged or n == c or n not in cluster_results:
                    continue
                
                n_times = cluster_results[n].get('weekday_times', [])
                if not n_times or any(t == float('inf') for t in n_times):
                    continue
                
                merge_candidates.append((c, n, c_times, n_times))
        
        # Controllo parallelo merge feasibility
        if merge_candidates:
            if verbose:
                print(f"  🔄 Controllo {len(merge_candidates)} coppie candidate in parallelo...")
            
            def check_merge_candidate(candidate):
                c, n, c_times, n_times = candidate
                can_merge, sum_times = self._can_merge_clusters(c_times, n_times, self.max_shift_time_min)
                return (c, n, can_merge, sum_times)
            
            with ThreadPoolExecutor(max_workers=self.n_cores) as executor:
                futures = [executor.submit(check_merge_candidate, candidate) 
                          for candidate in merge_candidates]
                
                for future in as_completed(futures):
                    c, n, can_merge, sum_times = future.result()
                    
                    if can_merge and c not in merged and n not in merged:
                        # Esegui merge
                        new_locs = new_cluster_dict[c] + new_cluster_dict[n]
                        new_cluster_dict[c] = new_locs
                        del new_cluster_dict[n]
                        merged.add(c)
                        merged.add(n)
                        merge_count += 1
                        
                        if verbose:
                            print(f"    ✅ Merged cluster {c} ({len(cluster_dict[c])} punti) + "
                                  f"cluster {n} ({len(cluster_dict[n])} punti) = "
                                  f"{len(new_locs)} punti")
        
        # Rinumera cluster per eliminare gap
        final_cluster_dict = {}
        for new_id, (old_id, location_ids) in enumerate(new_cluster_dict.items()):
            if len(location_ids) > 0:
                final_cluster_dict[new_id] = location_ids
        
        # Aggiorna mapping cluster nel DataFrame punti
        location_to_new_cluster = {}
        for new_cluster_id, location_ids in final_cluster_dict.items():
            for loc_id in location_ids:
                location_to_new_cluster[loc_id] = new_cluster_id
        
        updated_points = points_df.copy()
        updated_points['cluster'] = updated_points['location_id'].map(location_to_new_cluster)
        
        if verbose:
            print(f"  🎯 Completati {merge_count} merge. Cluster finali: {len(final_cluster_dict)}")
        
        return final_cluster_dict, updated_points
    
    def _smart_reclustering_strategy(self, problematic_clusters, cluster_dict, cluster_results):
        reclustering_plan = {}
        
        for cluster_id in problematic_clusters:
            cluster_size = cluster_results[cluster_id]['cluster_size']
            max_time = cluster_results[cluster_id]['max_time']
            
            if max_time == float('inf'):
                suggested_splits = 3
            else:
                time_ratio = max_time / self.max_shift_time_min
                size_factor = max(1, cluster_size / 50)
                suggested_splits = max(2, min(8, int(np.ceil(time_ratio * 1.2 + size_factor * 0.1))))
            
            max_feasible_splits = min(suggested_splits, cluster_size // 2)
            reclustering_plan[cluster_id] = max(2, max_feasible_splits)
        
        return reclustering_plan
    
    def run_optimized_clustering(self,
                                delivery_points: pd.DataFrame,
                                initial_k: int = 20,
                                max_iterations: int = 15,
                                time_limit_per_tsp: int = 3,
                                early_stopping_threshold: int = 3,
                                verbose: bool = False):
        
        print(f"🎯 Inizio clustering bilanciato ottimizzato per {len(delivery_points)} punti")
        
        self._load_cache()
        
        points = delivery_points.copy()
        scaler = StandardScaler()
        points_scaled = scaler.fit_transform(points[['lat', 'lon']])
        
        adaptive_k = max(initial_k, len(delivery_points) // 200)
        current_k = min(adaptive_k, len(delivery_points) // 5)
        
        print(f"📊 K iniziale adattivo: {current_k}")
        
        best_solution = None
        best_score = float('inf')
        iterations_without_improvement = 0
        
        total_start_time = time.time()
        
        for iteration in range(1, max_iterations + 1):
            iter_start = time.time()
            
            if verbose:
                print(f"\n🔄 Iterazione {iteration}/{max_iterations} - K = {current_k}")
            
            # K-means ottimizzato
            kmeans = KMeans(
                n_clusters=current_k, 
                random_state=42 + iteration,
                n_init=5,
                max_iter=100,
                tol=1e-3
            )
            
            cluster_labels = kmeans.fit_predict(points_scaled)
            points['cluster'] = cluster_labels
            
            # Crea dizionario cluster
            cluster_dict = {}
            for c in range(current_k):
                cluster_locations = points.loc[points['cluster'] == c, 'location_id'].tolist()
                if len(cluster_locations) > 0:
                    cluster_dict[c] = cluster_locations
            
            if verbose:
                sizes = [len(locs) for locs in cluster_dict.values()]
                print(f"  📏 Dimensioni cluster: min={min(sizes)}, max={max(sizes)}, media={np.mean(sizes):.1f}")
            
            # Valutazione performance ottimizzata
            cluster_results = self._parallel_cluster_evaluation_optimized(cluster_dict, time_limit_per_tsp)
            
            # *** NUOVA FASE: MERGE CLUSTER PICCOLI CONFINANTI ***
            if len(cluster_dict) > 5:  # Solo se ha senso fare merge
                neighbors_dict = self._find_neighboring_clusters_delaunay(points)
                cluster_dict, points = self._merge_small_neighboring_clusters(
                    cluster_dict, cluster_results, neighbors_dict, points, 
                    min_cluster_size=15, verbose=verbose
                )
                
                # Ricalcola risultati dopo merge
                if len(cluster_dict) != len(cluster_results):
                    if verbose:
                        print("  🔄 Ricalcolo performance dopo merge...")
                    cluster_results = self._parallel_cluster_evaluation_optimized(cluster_dict, time_limit_per_tsp)
            
            # Identifica problematici
            problematic_clusters = []
            cluster_stats = []
            
            for cluster_id, result in cluster_results.items():
                cluster_stats.append((cluster_id, result['cluster_size'], result['max_time']))
                if not result['feasible']:
                    problematic_clusters.append(cluster_id)
            
            # Valuta soluzione
            num_problematic = len(problematic_clusters)
            if num_problematic < best_score:
                best_score = num_problematic
                best_solution = cluster_dict.copy()
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
            
            iter_elapsed = time.time() - iter_start
            
            if verbose:
                problematic_stats = [(cid, size, time_min) for cid, size, time_min in cluster_stats 
                                   if cid in problematic_clusters]
                problematic_stats.sort(key=lambda x: x[2], reverse=True)
                
                print(f"  📊 Cluster problematici: {num_problematic}/{len(cluster_dict)}")
                print(f"  ⏱️ Tempo iterazione: {iter_elapsed:.1f}s")
                
                if problematic_stats:
                    print("  🔍 Top 5 cluster problematici:")
                    for cid, size, time_min in problematic_stats[:5]:
                        print(f"    Cluster {cid}: {size} punti, {time_min:.1f} min")
            
            # Condizioni di uscita
            if num_problematic <= early_stopping_threshold:
                print(f"✅ Early stopping: solo {num_problematic} cluster problematici")
                break
            
            if iterations_without_improvement >= 3:
                print(f"🔄 Nessun miglioramento per 3 iterazioni, fermata anticipata")
                break
            
            # Preparazione iterazione successiva - RECLUSTERING
            if iteration < max_iterations and num_problematic > 0:
                reclustering_plan = self._smart_reclustering_strategy(
                    problematic_clusters, cluster_dict, cluster_results
                )
                
                problematic_points_mask = points['cluster'].isin(problematic_clusters)
                problematic_points = points[problematic_points_mask].copy()
                good_points = points[~problematic_points_mask].copy()
                
                if len(problematic_points) > 0:
                    total_new_clusters = sum(reclustering_plan.values())
                    
                    if total_new_clusters < len(problematic_points):
                        problematic_scaled = scaler.transform(problematic_points[['lat', 'lon']])
                        
                        sub_kmeans = KMeans(
                            n_clusters=min(total_new_clusters, len(problematic_points)),
                            random_state=42 + iteration * 10,
                            n_init=3,
                            max_iter=50
                        )
                        
                        sub_labels = sub_kmeans.fit_predict(problematic_scaled)
                        
                        max_existing = good_points['cluster'].max() if len(good_points) > 0 else -1
                        problematic_points['cluster'] = sub_labels + max_existing + 1
                        
                        points = pd.concat([good_points, problematic_points], ignore_index=True)
                        current_k = points['cluster'].nunique()
                        
                        if verbose:
                            print(f"  🔧 Riclusterizzati {len(problematic_clusters)} → {total_new_clusters} nuovi cluster")
                    else:
                        current_k += len(problematic_clusters)
                        if verbose:
                            print(f"  📈 Incremento K globale: {current_k}")
        
        # Finalizzazione
        total_elapsed = time.time() - total_start_time
        self._save_cache()
        
        if best_solution is None:
            best_solution = cluster_dict
        
        # Rinumera cluster finale
        final_clusters = {}
        for new_id, (old_id, location_ids) in enumerate(best_solution.items()):
            if len(location_ids) > 0:
                final_clusters[new_id] = location_ids
        
        if verbose:
            print(f"\n🏁 COMPLETATO in {total_elapsed:.1f}s totali ({total_elapsed/60:.1f} minuti)")
            print(f"📊 Soluzione finale: {len(final_clusters)} cluster")
            
            sizes = [len(locs) for locs in final_clusters.values()]
            print(f"📏 Dimensioni cluster: min={min(sizes)}, max={max(sizes)}, media={np.mean(sizes):.1f}")
        
        # *** CALCOLA OUTPUT CON calc_clusters_stats ***
        if verbose:
            print("📊 Calcolo performance cluster finale con calc_clusters_stats...")
        
        clusters_list = list(final_clusters.values())
        
        performance_df = pc.calc_clusters_stats(
            clusters=clusters_list,
            time_limit=time_limit_per_tsp,
            parallel=True,
            max_workers=self.n_cores,
            verbose=verbose
        )
        
        return final_clusters, performance_df


# Funzione wrapper semplice
def run_optimized_balanced_clustering(delivery_points, 
                                     initial_k=None, 
                                     max_iterations=15, 
                                     n_cores=None):
    
    if initial_k is None:
        initial_k = max(10, len(delivery_points) // 150)
        initial_k = min(initial_k, 50)
        print(f"🎯 K iniziale stimato: {initial_k}")
    
    print(f"🎯 Dataset: {len(delivery_points)} punti")
    print(f"🎯 K iniziale: {initial_k}")
    
    clusterer = OptimizedBalancedClustering(
        max_shift_time_min=480,
        n_cores=n_cores
    )
    
    return clusterer.run_optimized_clustering(
        delivery_points=delivery_points,
        initial_k=initial_k,
        max_iterations=max_iterations
    )



cluster_dict, performance_df = run_optimized_balanced_clustering(pc.delivery_points, initial_k=50, n_cores=8, max_iterations=25)


🎯 Dataset: 3764 punti
🎯 K iniziale: 50
🚀 Inizializzato con 8 core CPU
🎯 Inizio clustering bilanciato ottimizzato per 3764 punti
📂 Caricata cache con 1875 entries
📊 K iniziale adattivo: 50
🔄 Calcolo performance di 50 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 993.4s
🔄 Calcolo performance di 44 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 1015.7s
✅ Early stopping: solo 3 cluster problematici
💾 Salvata cache con 1878 entries


### Salvataggio degli output

In [3]:
performance_df.to_csv("clustering_methods_performances/k-means_iterative_v2(k=50)_5.csv", index=False)

with open('cluster_dicts/cluster_dict_k_means_iter_v2(k=50)_5.pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)

# # Caricamento veloce  
# with open('cluster_dicts/cluster_dict_k_means_iter_v2.pkl', 'rb') as f:
#     cluster_dict = pickle.load(f)

# k50 AS

In [4]:
class OptimizedBalancedClustering:
    def __init__(self, 
                 max_shift_time_min: int = 480,
                 n_cores: int = None,
                 cache_dir: str = "./cluster_cache"):
        
        self.max_shift_time_min = max_shift_time_min
        self.n_cores = n_cores or max(1, mp.cpu_count() - 1)
        self.cache_dir = cache_dir
        self.global_cache = {}
        
        os.makedirs(cache_dir, exist_ok=True)
        print(f"🚀 Inizializzato con {self.n_cores} core CPU")
    
    def _cache_key(self, location_ids):
        return hash(tuple(sorted(location_ids)))
    
    def _load_cache(self):
        cache_file = os.path.join(self.cache_dir, "cluster_performance_cache.pkl")
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'rb') as f:
                    self.global_cache = pickle.load(f)
                print(f"📂 Caricata cache con {len(self.global_cache)} entries")
            except:
                self.global_cache = {}
    
    def _save_cache(self):
        cache_file = os.path.join(self.cache_dir, "cluster_performance_cache.pkl")
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(self.global_cache, f)
            print(f"💾 Salvata cache con {len(self.global_cache)} entries")
        except Exception as e:
            print(f"⚠️ Errore salvataggio cache: {e}")
    
    def _find_neighboring_clusters_delaunay(self, delivery_points):
        """
        Trova cluster confinanti usando triangolazione di Delaunay per ottimizzare velocità
        """
        try:
            coords = delivery_points[['lat', 'lon']].values
            tri = Delaunay(coords)
            
            neighbors_dict = {}
            for cluster_id in delivery_points['cluster'].unique():
                neighbors_dict[cluster_id] = set()
            
            # Per ogni triangolo, trova cluster coinvolti
            for triangle in tri.simplices:
                clusters_in_triangle = delivery_points.iloc[triangle]['cluster'].unique()
                
                if len(clusters_in_triangle) > 1:
                    for i, cluster1 in enumerate(clusters_in_triangle):
                        for j, cluster2 in enumerate(clusters_in_triangle):
                            if i != j:
                                neighbors_dict[cluster1].add(cluster2)
            
            # Converte set in list
            neighbors_dict = {k: list(v) for k, v in neighbors_dict.items()}
            return neighbors_dict
            
        except Exception as e:
            print(f"⚠️ Errore calcolo neighbors: {e}")
            return {}
    
    def _can_merge_clusters(self, times1, times2, threshold=480):
        """
        Verifica se due cluster possono essere uniti controllando che la somma 
        dei tempi medi per ogni giorno non superi il limite
        """
        if len(times1) != len(times2):
            return False, None
            
        sum_times = np.array(times1) + np.array(times2)
        return np.all(sum_times <= threshold), sum_times.tolist()
    
    def _parallel_cluster_evaluation_optimized(self, cluster_dict, time_limit=3):
        """
        Versione ottimizzata che usa calc_clusters_stats per evitare nested parallelism
        """
        print(f"🔄 Calcolo performance di {len(cluster_dict)} cluster con calc_clusters_stats...")
        start_time = time.time()
        
        # Filtra cluster non vuoti
        valid_clusters = {cid: locs for cid, locs in cluster_dict.items() if len(locs) > 0}
        
        if not valid_clusters:
            return {}
        
        # Usa calc_clusters_stats che è più efficiente per molti cluster
        clusters_list = list(valid_clusters.values())
        
        try:
            performance_df = pc.calc_clusters_stats_AS(
                clusters=clusters_list,
                time_limit=time_limit,
                parallel=True,
                max_workers=self.n_cores,
                verbose=False
            )
            
            # Converti risultati in formato compatibile
            results = {}
            cluster_ids = list(valid_clusters.keys())
            
            for i, cluster_id in enumerate(cluster_ids):
                cluster_data = performance_df[performance_df['cluster'] == f'Cluster {i+1}']
                
                if not cluster_data.empty:
                    max_time = cluster_data['mean_minutes'].max()
                    avg_time = cluster_data['mean_minutes'].mean()
                    
                    # Estrai tempi per giorno della settimana per merge analysis
                    weekday_times = []
                    for weekday in ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì']:
                        weekday_data = cluster_data[cluster_data['weekday'] == weekday]
                        if not weekday_data.empty:
                            weekday_times.append(weekday_data['mean_minutes'].iloc[0])
                        else:
                            weekday_times.append(0)
                    
                    # Aggiungi cache entry
                    cache_key = self._cache_key(valid_clusters[cluster_id])
                    result = {
                        'max_time': max_time,
                        'avg_time': avg_time,
                        'cluster_size': len(valid_clusters[cluster_id]),
                        'feasible': max_time <= self.max_shift_time_min + 30,
                        'weekday_times': weekday_times  # Nuovo campo per merge analysis
                    }
                    
                    self.global_cache[cache_key] = result
                    results[cluster_id] = result
                else:
                    results[cluster_id] = {
                        'max_time': float('inf'),
                        'avg_time': float('inf'),
                        'cluster_size': len(valid_clusters[cluster_id]),
                        'feasible': False,
                        'weekday_times': [float('inf')] * 5
                    }
            
            elapsed = time.time() - start_time
            print(f"✅ Completata valutazione ottimizzata in {elapsed:.1f}s")
            return results
            
        except Exception as e:
            print(f"⚠️ Errore in calc_clusters_stats: {e}")
            # Fallback al metodo originale
            return self._parallel_cluster_evaluation_fallback(cluster_dict, time_limit)
    
    def _parallel_cluster_evaluation_fallback(self, cluster_dict, time_limit=3):
        """Metodo fallback originale"""
        print(f"🔄 Fallback: calcolo performance di {len(cluster_dict)} cluster...")
        start_time = time.time()
        
        jobs = [(cid, loc_ids, time_limit) for cid, loc_ids in cluster_dict.items() if len(loc_ids) > 0]
        results = {}
        
        with ThreadPoolExecutor(max_workers=self.n_cores) as executor:
            future_to_cluster = {
                executor.submit(self._compute_cluster_performance_cached, loc_ids, time_limit): cid 
                for cid, loc_ids, time_limit in jobs
            }
            
            completed = 0
            for future in as_completed(future_to_cluster):
                cluster_id = future_to_cluster[future]
                try:
                    result = future.result()
                    results[cluster_id] = result
                    completed += 1
                    
                    if completed % 10 == 0:
                        elapsed = time.time() - start_time
                        print(f"  📊 Completati {completed}/{len(jobs)} cluster in {elapsed:.1f}s")
                        
                except Exception as e:
                    print(f"⚠️ Errore cluster {cluster_id}: {e}")
                    results[cluster_id] = {
                        'max_time': float('inf'),
                        'avg_time': float('inf'), 
                        'cluster_size': 0,
                        'feasible': False,
                        'weekday_times': [float('inf')] * 5
                    }
        
        elapsed = time.time() - start_time
        print(f"✅ Completata valutazione fallback in {elapsed:.1f}s")
        return results
    
    def _compute_cluster_performance_cached(self, location_ids, time_limit=3):
        cache_key = self._cache_key(location_ids)
        
        if cache_key in self.global_cache:
            return self.global_cache[cache_key]
        
        try:
            stats_df, _ = pc.single_cluster_stats_with_cache_AS(
                cluster_location_ids=location_ids,
                time_limit=time_limit,
                verbose=False,
                max_workers=1
            )
            
            if stats_df is not None and not stats_df.empty:
                max_time = stats_df['mean_minutes'].max()
                avg_time = stats_df['mean_minutes'].mean()
                
                # Estrai tempi per weekday per merge analysis
                weekday_times = []
                for weekday in ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì']:
                    weekday_data = stats_df[stats_df['weekday'] == weekday]
                    if not weekday_data.empty:
                        weekday_times.append(weekday_data['mean_minutes'].iloc[0])
                    else:
                        weekday_times.append(0)
                
                result = {
                    'max_time': max_time,
                    'avg_time': avg_time,
                    'cluster_size': len(location_ids),
                    'feasible': max_time <= self.max_shift_time_min + 30,
                    'weekday_times': weekday_times
                }
            else:
                result = {
                    'max_time': float('inf'),
                    'avg_time': float('inf'),
                    'cluster_size': len(location_ids),
                    'feasible': False,
                    'weekday_times': [float('inf')] * 5
                }
            
            self.global_cache[cache_key] = result
            return result
            
        except Exception as e:
            print(f"⚠️ Errore calcolo cluster {len(location_ids)} punti: {e}")
            return {
                'max_time': float('inf'),
                'avg_time': float('inf'),
                'cluster_size': len(location_ids),
                'feasible': False,
                'weekday_times': [float('inf')] * 5
            }
    
    def _merge_small_neighboring_clusters(self, cluster_dict, cluster_results, neighbors_dict, 
                                         points_df, min_cluster_size=15, verbose=True):
        """
        Unisce cluster piccoli confinanti se la somma dei tempi non supera il limite
        """
        if verbose:
            print(f"🔗 Analisi merge cluster piccoli (< {min_cluster_size} punti)...")
        
        merged = set()
        new_cluster_dict = cluster_dict.copy()
        merge_count = 0
        
        # Identifica cluster piccoli
        small_clusters = [c for c, locs in cluster_dict.items() 
                         if len(locs) <= min_cluster_size and c in cluster_results]
        
        if verbose:
            print(f"  📊 Trovati {len(small_clusters)} cluster piccoli da analizzare")
        
        # Prepara dati per controllo parallelo
        merge_candidates = []
        
        for c in small_clusters:
            if c in merged or c not in neighbors_dict:
                continue
                
            c_times = cluster_results[c].get('weekday_times', [])
            if not c_times or any(t == float('inf') for t in c_times):
                continue
            
            for n in neighbors_dict[c]:
                if n in merged or n == c or n not in cluster_results:
                    continue
                
                n_times = cluster_results[n].get('weekday_times', [])
                if not n_times or any(t == float('inf') for t in n_times):
                    continue
                
                merge_candidates.append((c, n, c_times, n_times))
        
        # Controllo parallelo merge feasibility
        if merge_candidates:
            if verbose:
                print(f"  🔄 Controllo {len(merge_candidates)} coppie candidate in parallelo...")
            
            def check_merge_candidate(candidate):
                c, n, c_times, n_times = candidate
                can_merge, sum_times = self._can_merge_clusters(c_times, n_times, self.max_shift_time_min)
                return (c, n, can_merge, sum_times)
            
            with ThreadPoolExecutor(max_workers=self.n_cores) as executor:
                futures = [executor.submit(check_merge_candidate, candidate) 
                          for candidate in merge_candidates]
                
                for future in as_completed(futures):
                    c, n, can_merge, sum_times = future.result()
                    
                    if can_merge and c not in merged and n not in merged:
                        # Esegui merge
                        new_locs = new_cluster_dict[c] + new_cluster_dict[n]
                        new_cluster_dict[c] = new_locs
                        del new_cluster_dict[n]
                        merged.add(c)
                        merged.add(n)
                        merge_count += 1
                        
                        if verbose:
                            print(f"    ✅ Merged cluster {c} ({len(cluster_dict[c])} punti) + "
                                  f"cluster {n} ({len(cluster_dict[n])} punti) = "
                                  f"{len(new_locs)} punti")
        
        # Rinumera cluster per eliminare gap
        final_cluster_dict = {}
        for new_id, (old_id, location_ids) in enumerate(new_cluster_dict.items()):
            if len(location_ids) > 0:
                final_cluster_dict[new_id] = location_ids
        
        # Aggiorna mapping cluster nel DataFrame punti
        location_to_new_cluster = {}
        for new_cluster_id, location_ids in final_cluster_dict.items():
            for loc_id in location_ids:
                location_to_new_cluster[loc_id] = new_cluster_id
        
        updated_points = points_df.copy()
        updated_points['cluster'] = updated_points['location_id'].map(location_to_new_cluster)
        
        if verbose:
            print(f"  🎯 Completati {merge_count} merge. Cluster finali: {len(final_cluster_dict)}")
        
        return final_cluster_dict, updated_points
    
    def _smart_reclustering_strategy(self, problematic_clusters, cluster_dict, cluster_results):
        reclustering_plan = {}
        
        for cluster_id in problematic_clusters:
            cluster_size = cluster_results[cluster_id]['cluster_size']
            max_time = cluster_results[cluster_id]['max_time']
            
            if max_time == float('inf'):
                suggested_splits = 3
            else:
                time_ratio = max_time / self.max_shift_time_min
                size_factor = max(1, cluster_size / 50)
                suggested_splits = max(2, min(8, int(np.ceil(time_ratio * 1.2 + size_factor * 0.1))))
            
            max_feasible_splits = min(suggested_splits, cluster_size // 2)
            reclustering_plan[cluster_id] = max(2, max_feasible_splits)
        
        return reclustering_plan
    
    def run_optimized_clustering(self,
                                delivery_points: pd.DataFrame,
                                initial_k: int = 20,
                                max_iterations: int = 15,
                                time_limit_per_tsp: int = 3,
                                early_stopping_threshold: int = 3,
                                verbose: bool = False):
        
        print(f"🎯 Inizio clustering bilanciato ottimizzato per {len(delivery_points)} punti")
        
        self._load_cache()
        
        points = delivery_points.copy()
        scaler = StandardScaler()
        points_scaled = scaler.fit_transform(points[['lat', 'lon']])
        
        adaptive_k = max(initial_k, len(delivery_points) // 200)
        current_k = min(adaptive_k, len(delivery_points) // 5)
        
        print(f"📊 K iniziale adattivo: {current_k}")
        
        best_solution = None
        best_score = float('inf')
        iterations_without_improvement = 0
        
        total_start_time = time.time()
        
        for iteration in range(1, max_iterations + 1):
            iter_start = time.time()
            
            if verbose:
                print(f"\n🔄 Iterazione {iteration}/{max_iterations} - K = {current_k}")
            
            # K-means ottimizzato
            kmeans = KMeans(
                n_clusters=current_k, 
                random_state=42 + iteration,
                n_init=5,
                max_iter=100,
                tol=1e-3
            )
            
            cluster_labels = kmeans.fit_predict(points_scaled)
            points['cluster'] = cluster_labels
            
            # Crea dizionario cluster
            cluster_dict = {}
            for c in range(current_k):
                cluster_locations = points.loc[points['cluster'] == c, 'location_id'].tolist()
                if len(cluster_locations) > 0:
                    cluster_dict[c] = cluster_locations
            
            if verbose:
                sizes = [len(locs) for locs in cluster_dict.values()]
                print(f"  📏 Dimensioni cluster: min={min(sizes)}, max={max(sizes)}, media={np.mean(sizes):.1f}")
            
            # Valutazione performance ottimizzata
            cluster_results = self._parallel_cluster_evaluation_optimized(cluster_dict, time_limit_per_tsp)
            
            # *** NUOVA FASE: MERGE CLUSTER PICCOLI CONFINANTI ***
            if len(cluster_dict) > 5:  # Solo se ha senso fare merge
                neighbors_dict = self._find_neighboring_clusters_delaunay(points)
                cluster_dict, points = self._merge_small_neighboring_clusters(
                    cluster_dict, cluster_results, neighbors_dict, points, 
                    min_cluster_size=15, verbose=verbose
                )
                
                # Ricalcola risultati dopo merge
                if len(cluster_dict) != len(cluster_results):
                    if verbose:
                        print("  🔄 Ricalcolo performance dopo merge...")
                    cluster_results = self._parallel_cluster_evaluation_optimized(cluster_dict, time_limit_per_tsp)
            
            # Identifica problematici
            problematic_clusters = []
            cluster_stats = []
            
            for cluster_id, result in cluster_results.items():
                cluster_stats.append((cluster_id, result['cluster_size'], result['max_time']))
                if not result['feasible']:
                    problematic_clusters.append(cluster_id)
            
            # Valuta soluzione
            num_problematic = len(problematic_clusters)
            if num_problematic < best_score:
                best_score = num_problematic
                best_solution = cluster_dict.copy()
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
            
            iter_elapsed = time.time() - iter_start
            
            if verbose:
                problematic_stats = [(cid, size, time_min) for cid, size, time_min in cluster_stats 
                                   if cid in problematic_clusters]
                problematic_stats.sort(key=lambda x: x[2], reverse=True)
                
                print(f"  📊 Cluster problematici: {num_problematic}/{len(cluster_dict)}")
                print(f"  ⏱️ Tempo iterazione: {iter_elapsed:.1f}s")
                
                if problematic_stats:
                    print("  🔍 Top 5 cluster problematici:")
                    for cid, size, time_min in problematic_stats[:5]:
                        print(f"    Cluster {cid}: {size} punti, {time_min:.1f} min")
            
            # Condizioni di uscita
            if num_problematic <= early_stopping_threshold:
                print(f"✅ Early stopping: solo {num_problematic} cluster problematici")
                break
            
            if iterations_without_improvement >= 3:
                print(f"🔄 Nessun miglioramento per 3 iterazioni, fermata anticipata")
                break
            
            # Preparazione iterazione successiva - RECLUSTERING
            if iteration < max_iterations and num_problematic > 0:
                reclustering_plan = self._smart_reclustering_strategy(
                    problematic_clusters, cluster_dict, cluster_results
                )
                
                problematic_points_mask = points['cluster'].isin(problematic_clusters)
                problematic_points = points[problematic_points_mask].copy()
                good_points = points[~problematic_points_mask].copy()
                
                if len(problematic_points) > 0:
                    total_new_clusters = sum(reclustering_plan.values())
                    
                    if total_new_clusters < len(problematic_points):
                        problematic_scaled = scaler.transform(problematic_points[['lat', 'lon']])
                        
                        sub_kmeans = KMeans(
                            n_clusters=min(total_new_clusters, len(problematic_points)),
                            random_state=42 + iteration * 10,
                            n_init=3,
                            max_iter=50
                        )
                        
                        sub_labels = sub_kmeans.fit_predict(problematic_scaled)
                        
                        max_existing = good_points['cluster'].max() if len(good_points) > 0 else -1
                        problematic_points['cluster'] = sub_labels + max_existing + 1
                        
                        points = pd.concat([good_points, problematic_points], ignore_index=True)
                        current_k = points['cluster'].nunique()
                        
                        if verbose:
                            print(f"  🔧 Riclusterizzati {len(problematic_clusters)} → {total_new_clusters} nuovi cluster")
                    else:
                        current_k += len(problematic_clusters)
                        if verbose:
                            print(f"  📈 Incremento K globale: {current_k}")
        
        # Finalizzazione
        total_elapsed = time.time() - total_start_time
        self._save_cache()
        
        if best_solution is None:
            best_solution = cluster_dict
        
        # Rinumera cluster finale
        final_clusters = {}
        for new_id, (old_id, location_ids) in enumerate(best_solution.items()):
            if len(location_ids) > 0:
                final_clusters[new_id] = location_ids
        
        if verbose:
            print(f"\n🏁 COMPLETATO in {total_elapsed:.1f}s totali ({total_elapsed/60:.1f} minuti)")
            print(f"📊 Soluzione finale: {len(final_clusters)} cluster")
            
            sizes = [len(locs) for locs in final_clusters.values()]
            print(f"📏 Dimensioni cluster: min={min(sizes)}, max={max(sizes)}, media={np.mean(sizes):.1f}")
        
        # *** CALCOLA OUTPUT CON calc_clusters_stats ***
        if verbose:
            print("📊 Calcolo performance cluster finale con calc_clusters_stats...")
        
        clusters_list = list(final_clusters.values())
        
        performance_df = pc.calc_clusters_stats_AS(
            clusters=clusters_list,
            time_limit=time_limit_per_tsp,
            parallel=True,
            max_workers=self.n_cores,
            verbose=verbose
        )
        
        return final_clusters, performance_df


# Funzione wrapper semplice
def run_optimized_balanced_clustering(delivery_points, 
                                     initial_k=None, 
                                     max_iterations=15, 
                                     n_cores=None):
    
    if initial_k is None:
        initial_k = max(10, len(delivery_points) // 150)
        initial_k = min(initial_k, 50)
        print(f"🎯 K iniziale stimato: {initial_k}")
    
    print(f"🎯 Dataset: {len(delivery_points)} punti")
    print(f"🎯 K iniziale: {initial_k}")
    
    clusterer = OptimizedBalancedClustering(
        max_shift_time_min=480,
        n_cores=n_cores
    )
    
    return clusterer.run_optimized_clustering(
        delivery_points=delivery_points,
        initial_k=initial_k,
        max_iterations=max_iterations
    )


import time
start = time.time()

cluster_dict, performance_df = run_optimized_balanced_clustering(pc.delivery_points_AS, initial_k=50, n_cores=8, max_iterations=25)

end = time.time()
print(f"Tempo di esecuzione algoritmo: {(end - start)/60:.2f} min")


🎯 Dataset: 2972 punti
🎯 K iniziale: 50
🚀 Inizializzato con 8 core CPU
🎯 Inizio clustering bilanciato ottimizzato per 2972 punti
📂 Caricata cache con 1878 entries
📊 K iniziale adattivo: 50
🔄 Calcolo performance di 50 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 369.1s
🔄 Calcolo performance di 42 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 353.2s
✅ Early stopping: solo 1 cluster problematici
💾 Salvata cache con 1881 entries
Tempo di esecuzione algoritmo: 17.70 min


### Salvataggio output

In [5]:
performance_df.to_csv("clustering_methods_performances/k-means_iterative_v2(k=50)_AS_5.csv", index=False)

with open('cluster_dicts/cluster_dict_k_means_iter_v2(k=50)_AS_5.pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)

# # Caricamento veloce  
# with open('cluster_dicts/cluster_dict_k_means_iter_v2.pkl', 'rb') as f:
#     cluster_dict = pickle.load(f)

# k50 ON

In [6]:
class OptimizedBalancedClustering:
    def __init__(self, 
                 max_shift_time_min: int = 480,
                 n_cores: int = None,
                 cache_dir: str = "./cluster_cache"):
        
        self.max_shift_time_min = max_shift_time_min
        self.n_cores = n_cores or max(1, mp.cpu_count() - 1)
        self.cache_dir = cache_dir
        self.global_cache = {}
        
        os.makedirs(cache_dir, exist_ok=True)
        print(f"🚀 Inizializzato con {self.n_cores} core CPU")
    
    def _cache_key(self, location_ids):
        return hash(tuple(sorted(location_ids)))
    
    def _load_cache(self):
        cache_file = os.path.join(self.cache_dir, "cluster_performance_cache.pkl")
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'rb') as f:
                    self.global_cache = pickle.load(f)
                print(f"📂 Caricata cache con {len(self.global_cache)} entries")
            except:
                self.global_cache = {}
    
    def _save_cache(self):
        cache_file = os.path.join(self.cache_dir, "cluster_performance_cache.pkl")
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(self.global_cache, f)
            print(f"💾 Salvata cache con {len(self.global_cache)} entries")
        except Exception as e:
            print(f"⚠️ Errore salvataggio cache: {e}")
    
    def _find_neighboring_clusters_delaunay(self, delivery_points):
        """
        Trova cluster confinanti usando triangolazione di Delaunay per ottimizzare velocità
        """
        try:
            coords = delivery_points[['lat', 'lon']].values
            tri = Delaunay(coords)
            
            neighbors_dict = {}
            for cluster_id in delivery_points['cluster'].unique():
                neighbors_dict[cluster_id] = set()
            
            # Per ogni triangolo, trova cluster coinvolti
            for triangle in tri.simplices:
                clusters_in_triangle = delivery_points.iloc[triangle]['cluster'].unique()
                
                if len(clusters_in_triangle) > 1:
                    for i, cluster1 in enumerate(clusters_in_triangle):
                        for j, cluster2 in enumerate(clusters_in_triangle):
                            if i != j:
                                neighbors_dict[cluster1].add(cluster2)
            
            # Converte set in list
            neighbors_dict = {k: list(v) for k, v in neighbors_dict.items()}
            return neighbors_dict
            
        except Exception as e:
            print(f"⚠️ Errore calcolo neighbors: {e}")
            return {}
    
    def _can_merge_clusters(self, times1, times2, threshold=480):
        """
        Verifica se due cluster possono essere uniti controllando che la somma 
        dei tempi medi per ogni giorno non superi il limite
        """
        if len(times1) != len(times2):
            return False, None
            
        sum_times = np.array(times1) + np.array(times2)
        return np.all(sum_times <= threshold), sum_times.tolist()
    
    def _parallel_cluster_evaluation_optimized(self, cluster_dict, time_limit=3):
        """
        Versione ottimizzata che usa calc_clusters_stats per evitare nested parallelism
        """
        print(f"🔄 Calcolo performance di {len(cluster_dict)} cluster con calc_clusters_stats...")
        start_time = time.time()
        
        # Filtra cluster non vuoti
        valid_clusters = {cid: locs for cid, locs in cluster_dict.items() if len(locs) > 0}
        
        if not valid_clusters:
            return {}
        
        # Usa calc_clusters_stats che è più efficiente per molti cluster
        clusters_list = list(valid_clusters.values())
        
        try:
            performance_df = pc.calc_clusters_stats_ON(
                clusters=clusters_list,
                time_limit=time_limit,
                parallel=True,
                max_workers=self.n_cores,
                verbose=False
            )
            
            # Converti risultati in formato compatibile
            results = {}
            cluster_ids = list(valid_clusters.keys())
            
            for i, cluster_id in enumerate(cluster_ids):
                cluster_data = performance_df[performance_df['cluster'] == f'Cluster {i+1}']
                
                if not cluster_data.empty:
                    max_time = cluster_data['mean_minutes'].max()
                    avg_time = cluster_data['mean_minutes'].mean()
                    
                    # Estrai tempi per giorno della settimana per merge analysis
                    weekday_times = []
                    for weekday in ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì']:
                        weekday_data = cluster_data[cluster_data['weekday'] == weekday]
                        if not weekday_data.empty:
                            weekday_times.append(weekday_data['mean_minutes'].iloc[0])
                        else:
                            weekday_times.append(0)
                    
                    # Aggiungi cache entry
                    cache_key = self._cache_key(valid_clusters[cluster_id])
                    result = {
                        'max_time': max_time,
                        'avg_time': avg_time,
                        'cluster_size': len(valid_clusters[cluster_id]),
                        'feasible': max_time <= self.max_shift_time_min + 30,
                        'weekday_times': weekday_times  # Nuovo campo per merge analysis
                    }
                    
                    self.global_cache[cache_key] = result
                    results[cluster_id] = result
                else:
                    results[cluster_id] = {
                        'max_time': float('inf'),
                        'avg_time': float('inf'),
                        'cluster_size': len(valid_clusters[cluster_id]),
                        'feasible': False,
                        'weekday_times': [float('inf')] * 5
                    }
            
            elapsed = time.time() - start_time
            print(f"✅ Completata valutazione ottimizzata in {elapsed:.1f}s")
            return results
            
        except Exception as e:
            print(f"⚠️ Errore in calc_clusters_stats: {e}")
            # Fallback al metodo originale
            return self._parallel_cluster_evaluation_fallback(cluster_dict, time_limit)
    
    def _parallel_cluster_evaluation_fallback(self, cluster_dict, time_limit=3):
        """Metodo fallback originale"""
        print(f"🔄 Fallback: calcolo performance di {len(cluster_dict)} cluster...")
        start_time = time.time()
        
        jobs = [(cid, loc_ids, time_limit) for cid, loc_ids in cluster_dict.items() if len(loc_ids) > 0]
        results = {}
        
        with ThreadPoolExecutor(max_workers=self.n_cores) as executor:
            future_to_cluster = {
                executor.submit(self._compute_cluster_performance_cached, loc_ids, time_limit): cid 
                for cid, loc_ids, time_limit in jobs
            }
            
            completed = 0
            for future in as_completed(future_to_cluster):
                cluster_id = future_to_cluster[future]
                try:
                    result = future.result()
                    results[cluster_id] = result
                    completed += 1
                    
                    if completed % 10 == 0:
                        elapsed = time.time() - start_time
                        print(f"  📊 Completati {completed}/{len(jobs)} cluster in {elapsed:.1f}s")
                        
                except Exception as e:
                    print(f"⚠️ Errore cluster {cluster_id}: {e}")
                    results[cluster_id] = {
                        'max_time': float('inf'),
                        'avg_time': float('inf'), 
                        'cluster_size': 0,
                        'feasible': False,
                        'weekday_times': [float('inf')] * 5
                    }
        
        elapsed = time.time() - start_time
        print(f"✅ Completata valutazione fallback in {elapsed:.1f}s")
        return results
    
    def _compute_cluster_performance_cached(self, location_ids, time_limit=3):
        cache_key = self._cache_key(location_ids)
        
        if cache_key in self.global_cache:
            return self.global_cache[cache_key]
        
        try:
            stats_df, _ = pc.single_cluster_stats_with_cache_ON(
                cluster_location_ids=location_ids,
                time_limit=time_limit,
                verbose=False,
                max_workers=1
            )
            
            if stats_df is not None and not stats_df.empty:
                max_time = stats_df['mean_minutes'].max()
                avg_time = stats_df['mean_minutes'].mean()
                
                # Estrai tempi per weekday per merge analysis
                weekday_times = []
                for weekday in ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì']:
                    weekday_data = stats_df[stats_df['weekday'] == weekday]
                    if not weekday_data.empty:
                        weekday_times.append(weekday_data['mean_minutes'].iloc[0])
                    else:
                        weekday_times.append(0)
                
                result = {
                    'max_time': max_time,
                    'avg_time': avg_time,
                    'cluster_size': len(location_ids),
                    'feasible': max_time <= self.max_shift_time_min + 30,
                    'weekday_times': weekday_times
                }
            else:
                result = {
                    'max_time': float('inf'),
                    'avg_time': float('inf'),
                    'cluster_size': len(location_ids),
                    'feasible': False,
                    'weekday_times': [float('inf')] * 5
                }
            
            self.global_cache[cache_key] = result
            return result
            
        except Exception as e:
            print(f"⚠️ Errore calcolo cluster {len(location_ids)} punti: {e}")
            return {
                'max_time': float('inf'),
                'avg_time': float('inf'),
                'cluster_size': len(location_ids),
                'feasible': False,
                'weekday_times': [float('inf')] * 5
            }
    
    def _merge_small_neighboring_clusters(self, cluster_dict, cluster_results, neighbors_dict, 
                                         points_df, min_cluster_size=15, verbose=True):
        """
        Unisce cluster piccoli confinanti se la somma dei tempi non supera il limite
        """
        if verbose:
            print(f"🔗 Analisi merge cluster piccoli (< {min_cluster_size} punti)...")
        
        merged = set()
        new_cluster_dict = cluster_dict.copy()
        merge_count = 0
        
        # Identifica cluster piccoli
        small_clusters = [c for c, locs in cluster_dict.items() 
                         if len(locs) <= min_cluster_size and c in cluster_results]
        
        if verbose:
            print(f"  📊 Trovati {len(small_clusters)} cluster piccoli da analizzare")
        
        # Prepara dati per controllo parallelo
        merge_candidates = []
        
        for c in small_clusters:
            if c in merged or c not in neighbors_dict:
                continue
                
            c_times = cluster_results[c].get('weekday_times', [])
            if not c_times or any(t == float('inf') for t in c_times):
                continue
            
            for n in neighbors_dict[c]:
                if n in merged or n == c or n not in cluster_results:
                    continue
                
                n_times = cluster_results[n].get('weekday_times', [])
                if not n_times or any(t == float('inf') for t in n_times):
                    continue
                
                merge_candidates.append((c, n, c_times, n_times))
        
        # Controllo parallelo merge feasibility
        if merge_candidates:
            if verbose:
                print(f"  🔄 Controllo {len(merge_candidates)} coppie candidate in parallelo...")
            
            def check_merge_candidate(candidate):
                c, n, c_times, n_times = candidate
                can_merge, sum_times = self._can_merge_clusters(c_times, n_times, self.max_shift_time_min)
                return (c, n, can_merge, sum_times)
            
            with ThreadPoolExecutor(max_workers=self.n_cores) as executor:
                futures = [executor.submit(check_merge_candidate, candidate) 
                          for candidate in merge_candidates]
                
                for future in as_completed(futures):
                    c, n, can_merge, sum_times = future.result()
                    
                    if can_merge and c not in merged and n not in merged:
                        # Esegui merge
                        new_locs = new_cluster_dict[c] + new_cluster_dict[n]
                        new_cluster_dict[c] = new_locs
                        del new_cluster_dict[n]
                        merged.add(c)
                        merged.add(n)
                        merge_count += 1
                        
                        if verbose:
                            print(f"    ✅ Merged cluster {c} ({len(cluster_dict[c])} punti) + "
                                  f"cluster {n} ({len(cluster_dict[n])} punti) = "
                                  f"{len(new_locs)} punti")
        
        # Rinumera cluster per eliminare gap
        final_cluster_dict = {}
        for new_id, (old_id, location_ids) in enumerate(new_cluster_dict.items()):
            if len(location_ids) > 0:
                final_cluster_dict[new_id] = location_ids
        
        # Aggiorna mapping cluster nel DataFrame punti
        location_to_new_cluster = {}
        for new_cluster_id, location_ids in final_cluster_dict.items():
            for loc_id in location_ids:
                location_to_new_cluster[loc_id] = new_cluster_id
        
        updated_points = points_df.copy()
        updated_points['cluster'] = updated_points['location_id'].map(location_to_new_cluster)
        
        if verbose:
            print(f"  🎯 Completati {merge_count} merge. Cluster finali: {len(final_cluster_dict)}")
        
        return final_cluster_dict, updated_points
    
    def _smart_reclustering_strategy(self, problematic_clusters, cluster_dict, cluster_results):
        reclustering_plan = {}
        
        for cluster_id in problematic_clusters:
            cluster_size = cluster_results[cluster_id]['cluster_size']
            max_time = cluster_results[cluster_id]['max_time']
            
            if max_time == float('inf'):
                suggested_splits = 3
            else:
                time_ratio = max_time / self.max_shift_time_min
                size_factor = max(1, cluster_size / 50)
                suggested_splits = max(2, min(8, int(np.ceil(time_ratio * 1.2 + size_factor * 0.1))))
            
            max_feasible_splits = min(suggested_splits, cluster_size // 2)
            reclustering_plan[cluster_id] = max(2, max_feasible_splits)
        
        return reclustering_plan
    
    def run_optimized_clustering(self,
                                delivery_points: pd.DataFrame,
                                initial_k: int = 20,
                                max_iterations: int = 15,
                                time_limit_per_tsp: int = 3,
                                early_stopping_threshold: int = 3,
                                verbose: bool = False):
        
        print(f"🎯 Inizio clustering bilanciato ottimizzato per {len(delivery_points)} punti")
        
        self._load_cache()
        
        points = delivery_points.copy()
        scaler = StandardScaler()
        points_scaled = scaler.fit_transform(points[['lat', 'lon']])
        
        adaptive_k = max(initial_k, len(delivery_points) // 200)
        current_k = min(adaptive_k, len(delivery_points) // 5)
        
        print(f"📊 K iniziale adattivo: {current_k}")
        
        best_solution = None
        best_score = float('inf')
        iterations_without_improvement = 0
        
        total_start_time = time.time()
        
        for iteration in range(1, max_iterations + 1):
            iter_start = time.time()
            
            if verbose:
                print(f"\n🔄 Iterazione {iteration}/{max_iterations} - K = {current_k}")
            
            # K-means ottimizzato
            kmeans = KMeans(
                n_clusters=current_k, 
                random_state=42 + iteration,
                n_init=5,
                max_iter=100,
                tol=1e-3
            )
            
            cluster_labels = kmeans.fit_predict(points_scaled)
            points['cluster'] = cluster_labels
            
            # Crea dizionario cluster
            cluster_dict = {}
            for c in range(current_k):
                cluster_locations = points.loc[points['cluster'] == c, 'location_id'].tolist()
                if len(cluster_locations) > 0:
                    cluster_dict[c] = cluster_locations
            
            if verbose:
                sizes = [len(locs) for locs in cluster_dict.values()]
                print(f"  📏 Dimensioni cluster: min={min(sizes)}, max={max(sizes)}, media={np.mean(sizes):.1f}")
            
            # Valutazione performance ottimizzata
            cluster_results = self._parallel_cluster_evaluation_optimized(cluster_dict, time_limit_per_tsp)
            
            # *** NUOVA FASE: MERGE CLUSTER PICCOLI CONFINANTI ***
            if len(cluster_dict) > 5:  # Solo se ha senso fare merge
                neighbors_dict = self._find_neighboring_clusters_delaunay(points)
                cluster_dict, points = self._merge_small_neighboring_clusters(
                    cluster_dict, cluster_results, neighbors_dict, points, 
                    min_cluster_size=15, verbose=verbose
                )
                
                # Ricalcola risultati dopo merge
                if len(cluster_dict) != len(cluster_results):
                    if verbose:
                        print("  🔄 Ricalcolo performance dopo merge...")
                    cluster_results = self._parallel_cluster_evaluation_optimized(cluster_dict, time_limit_per_tsp)
            
            # Identifica problematici
            problematic_clusters = []
            cluster_stats = []
            
            for cluster_id, result in cluster_results.items():
                cluster_stats.append((cluster_id, result['cluster_size'], result['max_time']))
                if not result['feasible']:
                    problematic_clusters.append(cluster_id)
            
            # Valuta soluzione
            num_problematic = len(problematic_clusters)
            if num_problematic < best_score:
                best_score = num_problematic
                best_solution = cluster_dict.copy()
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
            
            iter_elapsed = time.time() - iter_start
            
            if verbose:
                problematic_stats = [(cid, size, time_min) for cid, size, time_min in cluster_stats 
                                   if cid in problematic_clusters]
                problematic_stats.sort(key=lambda x: x[2], reverse=True)
                
                print(f"  📊 Cluster problematici: {num_problematic}/{len(cluster_dict)}")
                print(f"  ⏱️ Tempo iterazione: {iter_elapsed:.1f}s")
                
                if problematic_stats:
                    print("  🔍 Top 5 cluster problematici:")
                    for cid, size, time_min in problematic_stats[:5]:
                        print(f"    Cluster {cid}: {size} punti, {time_min:.1f} min")
            
            # Condizioni di uscita
            if num_problematic <= early_stopping_threshold:
                print(f"✅ Early stopping: solo {num_problematic} cluster problematici")
                break
            
            if iterations_without_improvement >= 3:
                print(f"🔄 Nessun miglioramento per 3 iterazioni, fermata anticipata")
                break
            
            # Preparazione iterazione successiva - RECLUSTERING
            if iteration < max_iterations and num_problematic > 0:
                reclustering_plan = self._smart_reclustering_strategy(
                    problematic_clusters, cluster_dict, cluster_results
                )
                
                problematic_points_mask = points['cluster'].isin(problematic_clusters)
                problematic_points = points[problematic_points_mask].copy()
                good_points = points[~problematic_points_mask].copy()
                
                if len(problematic_points) > 0:
                    total_new_clusters = sum(reclustering_plan.values())
                    
                    if total_new_clusters < len(problematic_points):
                        problematic_scaled = scaler.transform(problematic_points[['lat', 'lon']])
                        
                        sub_kmeans = KMeans(
                            n_clusters=min(total_new_clusters, len(problematic_points)),
                            random_state=42 + iteration * 10,
                            n_init=3,
                            max_iter=50
                        )
                        
                        sub_labels = sub_kmeans.fit_predict(problematic_scaled)
                        
                        max_existing = good_points['cluster'].max() if len(good_points) > 0 else -1
                        problematic_points['cluster'] = sub_labels + max_existing + 1
                        
                        points = pd.concat([good_points, problematic_points], ignore_index=True)
                        current_k = points['cluster'].nunique()
                        
                        if verbose:
                            print(f"  🔧 Riclusterizzati {len(problematic_clusters)} → {total_new_clusters} nuovi cluster")
                    else:
                        current_k += len(problematic_clusters)
                        if verbose:
                            print(f"  📈 Incremento K globale: {current_k}")
        
        # Finalizzazione
        total_elapsed = time.time() - total_start_time
        self._save_cache()
        
        if best_solution is None:
            best_solution = cluster_dict
        
        # Rinumera cluster finale
        final_clusters = {}
        for new_id, (old_id, location_ids) in enumerate(best_solution.items()):
            if len(location_ids) > 0:
                final_clusters[new_id] = location_ids
        
        if verbose:
            print(f"\n🏁 COMPLETATO in {total_elapsed:.1f}s totali ({total_elapsed/60:.1f} minuti)")
            print(f"📊 Soluzione finale: {len(final_clusters)} cluster")
            
            sizes = [len(locs) for locs in final_clusters.values()]
            print(f"📏 Dimensioni cluster: min={min(sizes)}, max={max(sizes)}, media={np.mean(sizes):.1f}")
        
        # *** CALCOLA OUTPUT CON calc_clusters_stats ***
        if verbose:
            print("📊 Calcolo performance cluster finale con calc_clusters_stats...")
        
        clusters_list = list(final_clusters.values())
        
        performance_df = pc.calc_clusters_stats_ON(
            clusters=clusters_list,
            time_limit=time_limit_per_tsp,
            parallel=True,
            max_workers=self.n_cores,
            verbose=verbose
        )
        
        return final_clusters, performance_df


# Funzione wrapper semplice
def run_optimized_balanced_clustering(delivery_points, 
                                     initial_k=None, 
                                     max_iterations=15, 
                                     n_cores=None):
    
    if initial_k is None:
        initial_k = max(10, len(delivery_points) // 150)
        initial_k = min(initial_k, 50)
        print(f"🎯 K iniziale stimato: {initial_k}")
    
    print(f"🎯 Dataset: {len(delivery_points)} punti")
    print(f"🎯 K iniziale: {initial_k}")
    
    clusterer = OptimizedBalancedClustering(
        max_shift_time_min=480,
        n_cores=n_cores
    )
    
    return clusterer.run_optimized_clustering(
        delivery_points=delivery_points,
        initial_k=initial_k,
        max_iterations=max_iterations
    )


import time
start = time.time()

cluster_dict, performance_df = run_optimized_balanced_clustering(pc.delivery_points_ON, initial_k=50, n_cores=8, max_iterations=25)

end = time.time()
print(f"Tempo di esecuzione algoritmo: {(end - start)/60:.2f} min")


🎯 Dataset: 3219 punti
🎯 K iniziale: 50
🚀 Inizializzato con 8 core CPU
🎯 Inizio clustering bilanciato ottimizzato per 3219 punti
📂 Caricata cache con 1881 entries
📊 K iniziale adattivo: 50
🔄 Calcolo performance di 50 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 372.3s
🔄 Calcolo performance di 45 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 365.1s
🔄 Calcolo performance di 51 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 434.8s
🔄 Calcolo performance di 46 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 443.3s
🔄 Calcolo performance di 55 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 484.4s
🔄 Calcolo performance di 46 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 470.0s
🔄 Calcolo performance di 58 cluster con calc_clusters_stats...
✅ Completata valutazione ottimizzata in 507.6s
🔄 Calcolo performance di 47 cluster con ca

### Salvataggio output

In [7]:
performance_df.to_csv("clustering_methods_performances/k-means_iterative_v2(k=50)_ON_5.csv", index=False)


with open('cluster_dicts/cluster_dict_k_means_iter_v2(k=50)_ON_5.pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)

# # Caricamento veloce  
# with open('cluster_dicts/cluster_dict_k_means_iter_v2.pkl', 'rb') as f:
#     cluster_dict = pickle.load(f)